<a href="https://colab.research.google.com/github/mikhailborok/FasolPromoBot/blob/main/FasolPromoBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ячейка для запуска бота

In [ ]:
# Исправленный код для запуска Telegram бота в Google Colab
import subprocess
import sys
import os
import asyncio
import nest_asyncio
import shutil
import importlib

# Разрешаем вложенные event loops для Colab
nest_asyncio.apply()

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        return True
    except:
        return False

# Устанавливаем зависимости
print("📦 Установка зависимостей...")
packages = ["python-telegram-bot==20.7", "qrcode[pil]", "opencv-python", "pyzbar", "nest-asyncio"]

for package in packages:
    if install_package(package):
        print(f"✅ {package}")
    else:
        print(f"❌ Ошибка установки {package}")

# Монтируем Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Переходим в папку бота
bot_folder = '/content/drive/MyDrive/FasolPromoBot'
os.chdir(bot_folder)

print(f"📂 Рабочая директория: {os.getcwd()}")

# ПОЛНАЯ ОЧИСТКА КЕША И МОДУЛЕЙ
print("🧹 Очистка кеша...")

# Удаляем __pycache__
if os.path.exists('__pycache__'):
    shutil.rmtree('__pycache__')
    print("✅ __pycache__ удален")

# КРИТИЧНО: Удаляем все наши модули из sys.modules
modules_to_remove = []
for module_name in list(sys.modules.keys()):
    if any(name in module_name for name in ['main', 'database', 'config', 'qr_handler']):
        modules_to_remove.append(module_name)

for module_name in modules_to_remove:
    if module_name in sys.modules:
        del sys.modules[module_name]
        print(f"🗑️ Удален модуль: {module_name}")

# Очищаем sys.path и добавляем заново
while bot_folder in sys.path:
    sys.path.remove(bot_folder)

sys.path.insert(0, bot_folder)
print(f"📁 Путь добавлен: {bot_folder}")

try:
    print("🔑 Проверка токена...")

    # Импортируем config
    import config
    print(f"✅ config импортирован")
    print(f"🔑 Токен найден: {config.BOT_TOKEN[:10]}...")

    print("🗄️ Инициализация БД...")
    import database
    # ИСПРАВЛЕНО: убран экранированный символ
    database.init_db()
    print("✅ База данных инициализирована")

    # ПРИНУДИТЕЛЬНЫЙ ПЕРЕИМПОРТ main
    print("📥 Принудительный импорт main...")

    # Если main уже в sys.modules - удаляем его
    if 'main' in sys.modules:
        del sys.modules['main']
        print("🗑️ Удален старый модуль main из памяти")

    # Импортируем main заново
    import main
    print("✅ main импортирован заново")

    # ПРИНУДИТЕЛЬНАЯ ПЕРЕЗАГРУЗКА МОДУЛЯ
    importlib.reload(main)
    print("🔄 main перезагружен")

    # Проверяем наличие нужных функций ПОСЛЕ перезагрузки
    required_functions = ['start', 'button_callback', 'handle_message']
    for func_name in required_functions:
        if hasattr(main, func_name):
            print(f"✅ Функция {func_name} найдена")
        else:
            print(f"❌ Функция {func_name} НЕ найдена")
            # Показываем что есть в модуле
            available = [attr for attr in dir(main) if not attr.startswith('_')]
            print(f"🔍 Доступные атрибуты в main: {available[:10]}...")
            raise AttributeError(f"Функция {func_name} не найдена в модуле main")

    print("📥 Импорт telegram компонентов...")
    from telegram.ext import Application, CommandHandler, CallbackQueryHandler, MessageHandler, filters

    # Создаем приложение
    print("🔧 Создание приложения...")
    application = Application.builder().token(config.BOT_TOKEN).build()

    # Регистрируем обработчики
    print("📋 Регистрация обработчиков...")
    application.add_handler(CommandHandler("start", main.start))
    application.add_handler(CallbackQueryHandler(main.button_callback))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, main.handle_message))

    # Проверяем, есть ли обработчик Web App
    if hasattr(main, 'handle_web_app_data'):
        application.add_handler(MessageHandler(filters.StatusUpdate.WEB_APP_DATA, main.handle_web_app_data))
        print("✅ Обработчик Web App добавлен")
    else:
        print("⚠️ Обработчик Web App не найден (нормально, если Mini App еще не настроен)")

    print("🚀 Запуск бота...")
    print("✅ Бот запущен! Найдите его в Telegram и протестируйте.")
    print("📋 Тестовые логины:")
    print("   👑 master master (мастер-админ)")
    print("   🏢 moscow_admin admin123 (админ Москвы)")
    print("   🏢 samara_admin admin456 (админ Самары)")
    print("⚠️ Для остановки: Runtime -> Interrupt execution")

    # Асинхронная функция запуска
    async def run_bot():
        await application.initialize()
        await application.start()
        await application.updater.start_polling()

        try:
            print("🔄 Бот работает...")
            while True:
                await asyncio.sleep(1)
        except KeyboardInterrupt:
            print("🛑 Получен сигнал остановки...")
        except Exception as e:
            print(f"❌ Ошибка в работе бота: {e}")
        finally:
            print("🧹 Остановка бота...")
            try:
                await application.updater.stop()
                await application.stop()
                await application.shutdown()
            except:
                pass
            print("✅ Бот остановлен")

    # Запускаем бота
    await run_bot()

except FileNotFoundError as e:
    print(f"❌ Файл не найден: {e}")
    print("Убедитесь, что все файлы находятся в папке /content/drive/MyDrive/FasolPromoBot/")
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")
    print("Проверьте содержимое файлов на наличие синтаксических ошибок")
    import traceback
    traceback.print_exc()
except AttributeError as e:
    print(f"❌ Ошибка атрибута: {e}")

    # Дополнительная диагностика для database
    try:
        print("\n🔍 Диагностика модуля database:")
        import database
        db_functions = [attr for attr in dir(database) if not attr.startswith('_')]
        print(f"📄 Функции в database: {db_functions}")

        # Проверяем конкретно init_db
        if hasattr(database, 'init_db'):
            print("✅ database.init_db найдена")
        else:
            print("❌ database.init_db НЕ найдена")
            # Ищем похожие функции
            init_funcs = [f for f in db_functions if 'init' in f.lower()]
            if init_funcs:
                print(f"🔍 Похожие функции: {init_funcs}")
    except Exception as diag_e:
        print(f"❌ Ошибка диагностики database: {diag_e}")

    import traceback
    traceback.print_exc()
except Exception as e:
    print(f"❌ Неожиданная ошибка: {e}")
    import traceback
    print("Подробная информация об ошибке:")
    traceback.print_exc()


📦 Установка зависимостей...
✅ python-telegram-bot==20.7
✅ qrcode[pil]
✅ opencv-python
✅ pyzbar
✅ nest-asyncio
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Рабочая директория: /content/drive/MyDrive/FasolPromoBot
🧹 Очистка кеша...
🗑️ Удален модуль: config
🗑️ Удален модуль: database
🗑️ Удален модуль: httpx._config
🗑️ Удален модуль: h2.config
🗑️ Удален модуль: configparser
🗑️ Удален модуль: attr._config
🗑️ Удален модуль: httpx._main
🗑️ Удален модуль: qrcode.main
🗑️ Удален модуль: main
📁 Путь добавлен: /content/drive/MyDrive/FasolPromoBot
🔑 Проверка токена...
✅ config импортирован
🔑 Токен найден: 7947587238...
🗄️ Инициализация БД...
✅ База данных инициализирована
✅ База данных инициализирована
📥 Принудительный импорт main...
✅ main импортирован заново
🔄 main перезагружен
✅ Функция start найдена
✅ Функция button_callback найдена
✅ Функция handle_message найдена
📥 Импорт telegram компонентов...
🔧 Создание п

ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_application.py", line 1234, in process_update
    await coroutine
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_basehandler.py", line 157, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/FasolPromoBot/main.py", line 839, in handle_message
    await my_coupons(update, context)
  File "/content/drive/MyDrive/FasolPromoBot/main.py", line 260, in my_coupons
    coupon_id, user_id_db, promo_id, created_at, redeemed, description, store_id = coupon
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 7)
